# Query the EFD for AuxTel acquisition pointing errors.

Craig Lage - 14-Nov-25

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst_efd_client import merge_packed_time_series as mpts
from lsst.summit.utils.efdUtils import calcNextDay
from matplotlib.backends.backend_pdf import PdfPages
import warnings
warnings.filterwarnings("ignore")

In [ ]:
client = makeEfdClient()

In [ ]:
dayObs=20251120
msg = getEfdData(client, "lsst.sal.Script.logevent_logMessage", dayObs=dayObs)
acq_msg = msg[msg['filePath'].str.contains('auxtel') & 
    msg['filePath'].str.contains('acquire')]
ptg_messages = acq_msg[acq_msg['message'].str.contains('radial pointing') &
    ~acq_msg['message'].str.contains('completed')]
print(f"There were {len(ptg_messages)} pointing acquisitions on {dayObs}.")
azs = []
els = []
errors = []
for i in range(len(ptg_messages)):
    error = float(ptg_messages['message'].iloc[i].split(' ')[5])
    t_start = Time(ptg_messages.index[i])
    t_end = t_start + TimeDelta(5.0, format='sec')
    mount_position = getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", begin=t_start, end=t_end)
    az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)['azimuthCalculatedAngle'].median()
    el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)['elevationCalculatedAngle'].median()
    azs.append(az)
    els.append(el)
    errors.append(error)

In [ ]:
times = ptg_messages['private_kafkaStamp'].values
dTs = []
for i in range(1, len(ptg_messages)):
    dT = times[i] - times[i-1]
    dTs.append(dT)
plt.hist(dTs, bins=50,  range=(0,1000))   

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,5))
plt.suptitle(f"Radial pointing errors in AuxTel acquisitions - {dayObs}")
axes[0].scatter(azs, errors)
axes[0].set_ylim(0, 200)
axes[0].set_xlabel("Azimuth")
axes[0].set_ylabel("Radial pointing error (arcseconds)")
axes[1].scatter(els, errors)
axes[1].set_ylim(0, 200)
axes[1].set_xlabel("Elevation")
axes[1].set_ylabel("Radial pointing error (arcseconds)")
plt.savefig(f"/home/cslage/DATA/AuxTel_Pointing_Error_{dayObs}.png")

In [ ]:
startDay = 20251101
endDay = 20251120
dayObs = startDay
pdf = PdfPages(f"/home/cslage/DATA/AuxTel_Pointing_Errors_20Nov25.pdf")


while dayObs <= endDay:
    try:
        msg = getEfdData(client, "lsst.sal.Script.logevent_logMessage", dayObs=dayObs)
        acq_msg = msg[msg['filePath'].str.contains('auxtel') & 
            msg['filePath'].str.contains('acquire')]
        ptg_messages = acq_msg[acq_msg['message'].str.contains('radial pointing') &
            ~acq_msg['message'].str.contains('completed')]
        print(f"There were {len(ptg_messages)} pointing acquisitions on {dayObs}.")
    except:
        dayObs = calcNextDay(dayObs)
        continue
    if len(ptg_messages) < 10:
        dayObs = calcNextDay(dayObs)
        continue
    azs = []
    els = []
    errors = []
    for i in range(len(ptg_messages)):
        error = float(ptg_messages['message'].iloc[i].split(' ')[5])
        t_start = Time(ptg_messages.index[i])
        t_end = t_start + TimeDelta(5.0, format='sec')
        mount_position = getEfdData(client, "lsst.sal.ATMCS.mount_AzEl_Encoders", begin=t_start, end=t_end)
        az = mpts(mount_position, 'azimuthCalculatedAngle', stride=1)['azimuthCalculatedAngle'].median()
        el = mpts(mount_position, 'elevationCalculatedAngle', stride=1)['elevationCalculatedAngle'].median()
        azs.append(az)
        els.append(el)
        errors.append(error)

    fig, axes = plt.subplots(1,2,figsize=(12,5))
    plt.suptitle(f"Radial pointing errors in AuxTel acquisitions - {dayObs}")
    axes[0].scatter(azs, errors)
    axes[0].set_ylim(0, 50)
    axes[0].set_xlabel("Azimuth")
    axes[0].set_ylabel("Radial pointing error (arcseconds)")
    axes[1].scatter(els, errors)
    axes[1].set_ylim(0, 50)
    axes[1].set_xlabel("Elevation")
    axes[1].set_ylabel("Radial pointing error (arcseconds)")
    pdf.savefig(fig)
    plt.clf()
    dayObs = calcNextDay(dayObs)
pdf.close()
